In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import pandas as pd
import numpy as np

In [3]:
url = "https://raw.githubusercontent.com/dranzerblaze/effect_std_drug/master/train.csv"
df = pd.read_csv(url)
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [4]:
df['name_of_drug'].value_counts()

Levonorgestrel                       724
Etonogestrel                         628
Ethinyl estradiol / norethindrone    547
Ethinyl estradiol / norgestimate     425
Nexplanon                            397
                                    ... 
Revia                                  1
Cetuximab                              1
Narcan Injection                       1
Gynazole-1                             1
Ogestrel-28                            1
Name: name_of_drug, Length: 2220, dtype: int64

In [5]:
df['use_case_for_drug'].value_counts()

Birth Control                                              5795
Depression                                                 1791
Pain                                                       1273
Anxiety                                                    1174
Acne                                                       1101
                                                           ... 
Cardiovascular Risk Reduction                                 1
Lactation Augmentation                                        1
Hyperlipoproteinemia Type III, Elevated beta-VLDL   IDL       1
Klinefelter Syndrome                                          1
Rejection Prophylaxis                                         1
Name: use_case_for_drug, Length: 636, dtype: int64

In [6]:
df.shape

(32165, 8)

In [0]:
unique_drugs = df['name_of_drug'].unique()

In [8]:
dict_drugs= {}
for i in range(len(unique_drugs)):
  dict_drugs[unique_drugs[i]] = i
df.name_of_drug	 = [dict_drugs[item] for item in df.name_of_drug]
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,0,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,1,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,2,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,3,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,4,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782


In [9]:
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [10]:

from nltk.sentiment.vader import SentimentIntensityAnalyzer
data_sample = "have a nice day"
sid = SentimentIntensityAnalyzer()
ss =sid.polarity_scores(data_sample)
ss

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


{'compound': 0.4215, 'neg': 0.0, 'neu': 0.417, 'pos': 0.583}

In [0]:
review_score = []
bhams = SentimentIntensityAnalyzer()
for i in range(df.shape[0]):
  review_text = df['review_by_patient'][i].lower()
  score_polarity = bhams.polarity_scores(review_text)
  review_score.append(score_polarity['compound'])


In [0]:
df['review_score'] = review_score

In [13]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_score
0,206461,0,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969,-0.2960
1,95260,1,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458,0.8603
2,92703,2,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969,0.7962
3,35696,3,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176,0.9403
4,155963,4,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782,-0.4342


In [14]:
df = df.drop(['review_by_patient'],axis=1)
df.head()

,patient_id,name_of_drug,use_case_for_drug,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_score
0,206461,0,Left Ventricular Dysfunction,9,20-May-12,27,8.022969,-0.2960
1,95260,1,ADHD,8,27-Apr-10,192,7.858458,0.8603
2,92703,2,Birth Control,5,14-Dec-09,17,6.341969,0.7962
3,35696,3,Opiate Dependence,9,27-Nov-16,37,6.590176,0.9403
4,155963,4,Benign Prostatic Hyperplasia,2,28-Nov-15,43,6.144782,-0.4342


In [0]:
df['drug_approved_by_UIC'] = pd.to_datetime(df['drug_approved_by_UIC'])

In [16]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_score
0,206461,0,Left Ventricular Dysfunction,9,2012-05-20,27,8.022969,-0.2960
1,95260,1,ADHD,8,2010-04-27,192,7.858458,0.8603
2,92703,2,Birth Control,5,2009-12-14,17,6.341969,0.7962
3,35696,3,Opiate Dependence,9,2016-11-27,37,6.590176,0.9403
4,155963,4,Benign Prostatic Hyperplasia,2,2015-11-28,43,6.144782,-0.4342


In [17]:
curr_date = pd.datetime.now().date()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


In [18]:
print (curr_date)

2020-04-08


In [0]:
curr_date = pd.to_datetime(curr_date)

In [0]:
df['day_diff'] = curr_date - df['drug_approved_by_UIC'] 

In [0]:
df['day_diff'] = df['day_diff']/np.timedelta64(1,'D')

In [22]:
df.head()

,patient_id,name_of_drug,use_case_for_drug,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_score,day_diff
0,206461,0,Left Ventricular Dysfunction,9,2012-05-20,27,8.022969,-0.2960,2880.0
1,95260,1,ADHD,8,2010-04-27,192,7.858458,0.8603,3634.0
2,92703,2,Birth Control,5,2009-12-14,17,6.341969,0.7962,3768.0
3,35696,3,Opiate Dependence,9,2016-11-27,37,6.590176,0.9403,1228.0
4,155963,4,Benign Prostatic Hyperplasia,2,2015-11-28,43,6.144782,-0.4342,1593.0


In [23]:
df = df.drop(['drug_approved_by_UIC','patient_id'],axis=1)
df.head()

,name_of_drug,use_case_for_drug,effectiveness_rating,number_of_times_prescribed,base_score,review_score,day_diff
0,0,Left Ventricular Dysfunction,9,27,8.022969,-0.2960,2880.0
1,1,ADHD,8,192,7.858458,0.8603,3634.0
2,2,Birth Control,5,17,6.341969,0.7962,3768.0
3,3,Opiate Dependence,9,37,6.590176,0.9403,1228.0
4,4,Benign Prostatic Hyperplasia,2,43,6.144782,-0.4342,1593.0


In [0]:
df = pd.get_dummies(df)

In [25]:
df.shape

(32165, 642)

In [26]:
df.head()

,name_of_drug,effectiveness_rating,number_of_times_prescribed,base_score,review_score,day_diff,use_case_for_drug_0</span> users found this comment helpful.,use_case_for_drug_10</span> users found this comment helpful.,use_case_for_drug_11</span> users found this comment helpful.,use_case_for_drug_12</span> users found this comment helpful.,use_case_for_drug_13</span> users found this comment helpful.,use_case_for_drug_14</span> users found this comment helpful.,use_case_for_drug_15</span> users found this comment helpful.,use_case_for_drug_16</span> users found this comment helpful.,use_case_for_drug_17</span> users found this comment helpful.,use_case_for_drug_18</span> users found this comment helpful.,use_case_for_drug_19</span> users found this comment helpful.,use_case_for_drug_1</span> users found this comment helpful.,use_case_for_drug_20</span> users found this comment helpful.,use_case_for_drug_21</span> users found this comment helpful.,use_case_for_drug_23</span> users found this comment helpful.,use_case_for_drug_24</span> users found this comment helpful.,use_case_for_drug_25</span> users found this comment helpful.,use_case_for_drug_27</span> users found this comment helpful.,use_case_for_drug_28</span> users found this comment helpful.,use_case_for_drug_2</span> users found this comment helpful.,use_case_for_drug_33</span> users found this comment helpful.,use_case_for_drug_34</span> users found this comment helpful.,use_case_for_drug_35</span> users found this comment helpful.,use_case_for_drug_3</span> users found this comment helpful.,use_case_for_drug_42</span> users found this comment helpful.,use_case_for_drug_43</span> users found this comment helpful.,use_case_for_drug_4</span> users found this comment helpful.,use_case_for_drug_5</span> users found this comment helpful.,use_case_for_drug_61</span> users found this comment helpful.,use_case_for_drug_63</span> users found this comment helpful.,use_case_for_drug_6</span> users found this comment helpful.,use_case_for_drug_75</span> users found this comment helpful.,use_case_for_drug_79</span> users found this comment helpful.,use_case_for_drug_7</span> users found this comment helpful.,...,use_case_for_drug_Vaginal Yeast Infection,use_case_for_drug_Varicella-Zoste,use_case_for_drug_Varicose Veins,use_case_for_drug_Ventricular Fibrillation,use_case_for_drug_Ventricular Tachycardia,use_case_for_drug_Vertig,use_case_for_drug_Vitamin B12 Deficiency,use_case_for_drug_Vitamin D Deficiency,use_case_for_drug_Vitamin K Deficiency,use_case_for_drug_Vitamin/Mineral Supplementation and Deficiency,use_case_for_drug_Vitamin/Mineral Supplementation during Pregnancy/Lactation,use_case_for_drug_Vulvodynia,use_case_for_drug_Warts,use_case_for_drug_Wegener's Granulomatosis,use_case_for_drug_Weight Loss,use_case_for_drug_Women (minoxidil),use_case_for_drug_Xerostomia,use_case_for_drug_acial Wrinkles,use_case_for_drug_actor IX Deficiency,use_case_for_drug_ailure to Thrive,use_case_for_drug_amilial Cold Autoinflammatory Syndrome,use_case_for_drug_amilial Mediterranean Feve,use_case_for_drug_atigue,use_case_for_drug_emale Infertility,use_case_for_drug_eve,use_case_for_drug_ge (amlodipine / valsartan),use_case_for_drug_ge HCT (amlodipine / hydrochlorothiazide / valsartan),use_case_for_drug_ibromyalgia,use_case_for_drug_lic Acid Deficiency,use_case_for_drug_mance Anxiety,use_case_for_drug_min / pioglitazone),use_case_for_drug_min / rosiglitazone),use_case_for_drug_min / saxagliptin),use_case_for_drug_min / sitagliptin),use_case_for_drug_min),use_case_for_drug_moterol / mometasone),use_case_for_drug_moterol),use_case_for_drug_mulation) (phenylephrine),use_case_for_drug_von Willebrand's Disease,use_case_for_drug_zen Shoulde
0,0,9,27,8.022969,-0.2960,2880.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,8,192,7.858458,0.8603,3634.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
df.columns[3]

'base_score'

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
values_df = df.values
scaled = scaler.fit_transform(values_df)

In [0]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	dff = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(dff.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(dff.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [30]:
reframed = series_to_supervised(scaled, 1, 1)
reframed.head(10)

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var18(t-1),var19(t-1),var20(t-1),var21(t-1),var22(t-1),var23(t-1),var24(t-1),var25(t-1),var26(t-1),var27(t-1),var28(t-1),var29(t-1),var30(t-1),var31(t-1),var32(t-1),var33(t-1),var34(t-1),var35(t-1),var36(t-1),var37(t-1),var38(t-1),var39(t-1),var40(t-1),...,var603(t),var604(t),var605(t),var606(t),var607(t),var608(t),var609(t),var610(t),var611(t),var612(t),var613(t),var614(t),var615(t),var616(t),var617(t),var618(t),var619(t),var620(t),var621(t),var622(t),var623(t),var624(t),var625(t),var626(t),var627(t),var628(t),var629(t),var630(t),var631(t),var632(t),var633(t),var634(t),var635(t),var636(t),var637(t),var638(t),var639(t),var640(t),var641(t),var642(t)
1,-0.910130,0.611070,-0.024937,0.855246,-0.372152,0.778772,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.117768,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658
2,-0.907793,0.305701,4.581624,0.751694,1.331437,1.546318,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.117768,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658
3,-0.905457,-0.610405,-0.304123,-0.202868,1.236998,1.682725,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.117768,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658
4,-0.903120,0.611070,0.254248,-0.046634,1.449302,-0.902907,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.117768,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.02

In [31]:
reframed.columns[645]

'var4(t)'

In [0]:
lst = []
for i in range(0,642):
  lst.append(i)
lst.append(645)

In [33]:
reframed1=reframed.iloc[:,lst]
reframed1.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var18(t-1),var19(t-1),var20(t-1),var21(t-1),var22(t-1),var23(t-1),var24(t-1),var25(t-1),var26(t-1),var27(t-1),var28(t-1),var29(t-1),var30(t-1),var31(t-1),var32(t-1),var33(t-1),var34(t-1),var35(t-1),var36(t-1),var37(t-1),var38(t-1),var39(t-1),var40(t-1),...,var604(t-1),var605(t-1),var606(t-1),var607(t-1),var608(t-1),var609(t-1),var610(t-1),var611(t-1),var612(t-1),var613(t-1),var614(t-1),var615(t-1),var616(t-1),var617(t-1),var618(t-1),var619(t-1),var620(t-1),var621(t-1),var622(t-1),var623(t-1),var624(t-1),var625(t-1),var626(t-1),var627(t-1),var628(t-1),var629(t-1),var630(t-1),var631(t-1),var632(t-1),var633(t-1),var634(t-1),var635(t-1),var636(t-1),var637(t-1),var638(t-1),var639(t-1),var640(t-1),var641(t-1),var642(t-1),var4(t)
1,-0.910130,0.611070,-0.024937,0.855246,-0.372152,0.778772,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658,0.751694
2,-0.907793,0.305701,4.581624,0.751694,1.331437,1.546318,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658,-0.202868
3,-0.905457,-0.610405,-0.304123,-0.202868,1.236998,1.682725,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,-0.005576,-0.005576,-0.007886,-0.009658,-0.018496,-0.009658,-0.028985,-0.007886,-0.009658,-0.009658,-0.046634
4,-0.903120,0.611070,0.254248,-0.046634,1.449302,-0.902907,-0.024312,-0.012469,-0.007886,-0.009658,-0.012469,-0.011152,-0.007886,-0.009658,-0.011152,-0.009658,-0.011152,-0.01673,-0.009658,-0.012469,-0.007886,-0.007886,-0.007886,-0.005576,-0.007886,-0.0216,-0.007886,-0.005576,-0.005576,-0.023663,-0.005576,-0.005576,-0.019319,-0.014754,-0.005576,-0.005576,-0.019319,-0.005576,-0.005576,-0.015773,...,-0.005576,-0.005576,-0.007886,-0.013659,-0.022309,-0.007886,-0.007886,-0.005576,-0.011152,-0.020867,-0.012469,-0.005576,-0.007886,-0.150995,-0.005576,-0.012469,-0.015773,-0.005576,-0.007886,-0.007886,-0.005576,-0.018496,-0.024944,-0.011152,-0.012469,-0.005576,-0.103667,-0.005576,-0.037011,

In [49]:
values = reframed1.values

n_train_time = 31000
train = values[:n_train_time, :]
test = values[n_train_time:, :]
##test = values[n_train_time:n_test_time, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features].

(31000, 1, 642) (31000,) (1164, 1, 642) (1164,)


In [0]:
import keras
from tensorflow.python.client import device_lib
from collections import defaultdict
import re
import sys
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model

In [51]:
model = Sequential()
model.add(LSTM(200, input_shape=(train_X.shape[1], train_X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=30, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Train on 31000 samples, validate on 1164 samples
Epoch 1/30
 - 11s - loss: 1.0097 - val_loss: 0.9047
Epoch 2/30
 - 9s - loss: 0.9915 - val_loss: 0.9204
Epoch 3/30
 - 9s - loss: 0.9766 - val_loss: 0.9446
Epoch 4/30
 - 9s - loss: 0.9694 - val_loss: 0.9468
Epoch 5/30
 - 9s - loss: 0.9649 - val_loss: 0.9629
Epoch 6/30
 - 9s - loss: 0.9614 - val_loss: 0.9677
Epoch 7/30
 - 9s - loss: 0.9578 - val_loss: 0.9763
Epoch 8/30
 - 9s - loss: 0.9532 - val_loss: 0.9832
Epoch 9/30
 - 9s - loss: 0.9485 - val_loss: 0.9972
Epoch 10/30
 - 9s - loss: 0.9450 - val_loss: 1.0046
Epoch 11/30
 - 9s - loss: 0.9409 - val_loss: 1.0115
Epoch 12/30
 - 9s - loss: 0.9344 - val_loss: 1.0098
Epoch 13/30
 - 9s - loss: 0.9287 - val_loss: 1.0317
Epoch 14/30
 - 9s - loss: 0.9243 - val_loss: 1.0470
Epoch 15/30
 - 8s - loss: 0.9197 - val_loss: 1.0475
Epoch 16/30
 - 9s - loss: 0.9135 - val_loss: 1.0404
Epoch 17/30
 - 9s - loss: 0.9078 - val_loss: 1.0529
Epoch 18/30
 - 9s - loss: 0.9033 - val_loss: 1.0599
Epoch 19/30
 - 9s - los